# IMPORT :

In [1]:
import os
import re
import gensim.downloader as api
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Download some NLP models for processing, optional

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/hbar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hbar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# PRE-PROCESS :

Load GloVe model with Gensim's API

In [3]:
# 200-dimensional GloVe embeddings
embeddings_model = api.load("glove-twitter-200")

Function to compute the average word vector for a tweet

In [4]:
def get_avg_embedding(tweet, model, vector_size=200):
    words = tweet.split()  # Tokenize by whitespace
    word_vectors = [model[word] for word in words if word in model]
    if not word_vectors:  # If no words in the tweet are in the vocabulary, return a zero vector
        return np.zeros(vector_size)
    return np.mean(word_vectors, axis=0)

Basic preprocessing function

In [5]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenization
    words = text.split()
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

Read all training files and concatenate them into one dataframe

In [6]:
li = []
for filename in os.listdir("train_tweets"):
    df = pd.read_csv("train_tweets/" + filename)
    li.append(df)
df = pd.concat(li, ignore_index=True)

Apply preprocessing to each tweet

In [7]:
df['Tweet'] = df['Tweet'].apply(preprocess_text)

Apply preprocessing to each tweet and obtain vectors

In [8]:
vector_size = 200  # Adjust based on the chosen GloVe model
tweet_vectors = np.vstack([get_avg_embedding(tweet, embeddings_model, vector_size) for tweet in df['Tweet']])
tweet_df = pd.DataFrame(tweet_vectors)

Attach the vectors into the original dataframe

In [9]:
period_features = pd.concat([df, tweet_df], axis=1)

Drop the columns that are not useful anymore

In [ ]:
period_features = period_features.drop(columns=['Timestamp', 'Tweet'])

Group the tweets into their corresponding periods. This way we generate an average embedding vector for each period

In [ ]:
period_features = period_features.groupby(['MatchID', 'PeriodID', 'ID']).mean().reset_index()

We drop the non-numerical features and keep the embeddings values for each period

In [ ]:
X = period_features.drop(columns=['EventType', 'MatchID', 'PeriodID', 'ID']).values

We extract the labels of our training samples

In [ ]:
y = period_features['EventType'].values

# Evaluating on a test set :

We split our data into a training and test set that we can use to train our classifier without fine-tuning into the validation set and without submitting too many times into Kaggle

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

We set up a basic classifier that we train and then calculate the accuracy on our test set

In [ ]:
clf = LogisticRegression(random_state=42, max_iter=1000).fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
print("Test set: ", accuracy_score(y_test, y_pred))

# For Kaggle submission :

This time we train our classifier on the full dataset that it is available to us.


In [ ]:
clf = LogisticRegression(random_state=42, max_iter=1000).fit(X, y)

We add a dummy classifier for sanity purposes

In [ ]:
dummy_clf = DummyClassifier(strategy="most_frequent").fit(X, y)

In [ ]:
predictions = []
dummy_predictions = []

We read each file separately, we preprocess the tweets and then use the classifier to predict the labels.
Finally, we concatenate all predictions into a list that will eventually be concatenated and exported to be submitted on Kaggle.

In [ ]:
for fname in os.listdir("eval_tweets"):
    val_df = pd.read_csv("eval_tweets/" + fname)
    val_df['Tweet'] = val_df['Tweet'].apply(preprocess_text)

    tweet_vectors = np.vstack([get_avg_embedding(tweet, embeddings_model, vector_size) for tweet in val_df['Tweet']])
    tweet_df = pd.DataFrame(tweet_vectors)

    period_features = pd.concat([val_df, tweet_df], axis=1)
    period_features = period_features.drop(columns=['Timestamp', 'Tweet'])
    period_features = period_features.groupby(['MatchID', 'PeriodID', 'ID']).mean().reset_index()
    X = period_features.drop(columns=['MatchID', 'PeriodID', 'ID']).values

    preds = clf.predict(X)
    dummy_preds = dummy_clf.predict(X)

    period_features['EventType'] = preds
    period_features['DummyEventType'] = dummy_preds

    predictions.append(period_features[['ID', 'EventType']])
    dummy_predictions.append(period_features[['ID', 'DummyEventType']])

In [ ]:
pred_df = pd.concat(predictions)
pred_df.to_csv('logistic_predictions.csv', index=False)

In [ ]:
pred_df = pd.concat(dummy_predictions)
pred_df.to_csv('dummy_predictions.csv', index=False)